In [9]:
from pathlib import Path
import pandas as pd
from scipy.stats import skew, kurtosis
import numpy as np, matplotlib.pyplot as plt, seaborn as sns
from typing import Literal
from sklearn.preprocessing import MinMaxScaler,StandardScaler

data_path = Path("../data/processed/S&P500_stock_hist_5y.parquet")
df = pd.read_parquet(data_path)

In [10]:
#reuseable function
def standardize_bygroup(df:pd.DataFrame,columns=None):
    df_copy = df.copy()
    if columns is None:
        columns = df_copy.select_dtypes(include=np.number).columns
    df_copy[columns] = StandardScaler().fit_transform(df_copy[columns])
    return df_copy

In [11]:
df = df[df['ticker'].isin(['NVDA'])]
# df = df.groupby('ticker', group_keys=False).apply(standardize_bygroup)
df = df.reset_index(drop=True)

## Implement feature engineering

In [16]:
#cumulative return over the last five trading days
df["mom_5"] = df["adj close"]/df["adj close"].shift(5)-1
df["log_return"] = np.log(df["adj close"]).diff()
df["vol_std_5"] = df["log_return"].rolling(5).std()
df.head(10)

Price,date,adj close,close,high,low,open,volume,ticker,daily_rtn,excess_return,mom_5,log_return,vol_std_5
0,2020-08-20,12.101113,12.14100,12.37500,11.87875,11.97500,921388000,NVDA,0.000206,-0.002953,NaN,NaN,NaN
1,2020-08-21,12.641834,12.68350,12.80875,12.19525,12.20175,999868000,NVDA,0.044684,0.041242,NaN,0.043714,NaN
2,2020-08-24,12.678459,12.72025,12.91250,12.50750,12.88375,490564000,NVDA,0.002897,-0.007147,NaN,0.002893,NaN
3,2020-08-25,12.708115,12.75000,12.76125,12.57375,12.63075,289076000,NVDA,0.002339,-0.001257,NaN,0.002336,NaN
4,2020-08-26,12.731039,12.77300,12.86850,12.67775,12.79925,321244000,NVDA,0.001804,-0.008392,NaN,0.001802,NaN
5,2020-08-27,12.586762,12.62825,12.85000,12.56650,12.78150,317604000,NVDA,-0.011333,-0.013006,0.040133,-0.011397,0.020906
6,2020-08-28,13.104554,13.14775,13.14800,12.64250,12.67500,537160000,NVDA,0.041138,0.034405,0.036602,0.040314,0.019453
7,2020-08-31,13.330562,13.37450,13.57500,13.03775,13.18275,500840000,NVDA,0.017246,0.019441,0.051434,0.017099,0.019705
8,2020-09-01,13.779715,13.82100,13.99375,13.43650,13.48000,511316000,NVDA,0.033693,0.026168,0.084324,0.033138,0.021437
9,2020-09-02,14.303645,14.34650,14.72675,13.90000,14.70375,874012000,NVDA,0.038022,0.022656,0.123525,0.037317,0.021361


**Rationale:**
- calculate return in the next 5 days
